In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

In [3]:
# Carregar os dados
df = pd.read_csv("/content/previsao_de_renda.csv")

In [4]:
# Preparar os dados
X = df.drop('renda', axis=1)
y = df['renda']

In [5]:
# Converter variáveis categóricas em dummies
X = pd.get_dummies(X, drop_first=True)

In [6]:
# Separar em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [7]:
# Escalar os dados
scaler = StandardScaler()

# Ajustar e transformar o escalonador nos dados de treinamento
X_train_scaled = scaler.fit_transform(X_train)

# Transforme os dados de teste usando o mesmo scaler
X_test_scaled = scaler.transform(X_test)

# Verifique os valores NaN nos dados dimensionados
print(np.isnan(X_train_scaled).any())
print(np.isnan(X_test_scaled).any())

# Imputar valores NaN com a média de cada coluna - isso substituirá quaisquer NaNs que apareceram durante o dimensionamento
!pip install sklearn
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
X_train_scaled = imputer.fit_transform(X_train_scaled)
X_test_scaled = imputer.transform(X_test_scaled)

# Verifique novamente os valores NaN nos dados dimensionados para confirmar a imputação
print(np.isnan(X_train_scaled).any())
print(np.isnan(X_test_scaled).any())

True
True
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
False
False


In [8]:
# 2. Regularização Ridge
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
ridge_scores = []

for alpha in alphas:
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train_scaled, y_train)
    y_pred = ridge.predict(X_test_scaled)
    score = r2_score(y_test, y_pred)
    ridge_scores.append(score)
    print(f"Ridge (alpha={alpha}): R² = {score:.4f}")

best_ridge_alpha = alphas[np.argmax(ridge_scores)]
print(f"\nMelhor modelo Ridge: alpha = {best_ridge_alpha}, R² = {max(ridge_scores):.4f}")

Ridge (alpha=0): R² = 0.2681
Ridge (alpha=0.001): R² = 0.2681
Ridge (alpha=0.005): R² = 0.2681
Ridge (alpha=0.01): R² = 0.2681
Ridge (alpha=0.05): R² = 0.2681
Ridge (alpha=0.1): R² = 0.2681

Melhor modelo Ridge: alpha = 0, R² = 0.2681


In [9]:
# 3. Regularização Lasso
lasso_scores = []

for alpha in alphas:
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train_scaled, y_train)
    y_pred = lasso.predict(X_test_scaled)
    score = r2_score(y_test, y_pred)
    lasso_scores.append(score)
    print(f"Lasso (alpha={alpha}): R² = {score:.4f}")

best_lasso_alpha = alphas[np.argmax(lasso_scores)]
print(f"\nMelhor modelo Lasso: alpha = {best_lasso_alpha}, R² = {max(lasso_scores):.4f}")

# Comparação Ridge vs Lasso
if max(ridge_scores) > max(lasso_scores):
    print("\nRidge obteve melhor resultado.")
else:
    print("\nLasso obteve melhor resultado.")


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.855e+11, tolerance: 7.723e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent

Lasso (alpha=0): R² = 0.2681


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.833e+11, tolerance: 7.723e+07
  model = cd_fast.enet_coordinate_descent(


Lasso (alpha=0.001): R² = 0.2681


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.746e+11, tolerance: 7.723e+07
  model = cd_fast.enet_coordinate_descent(


...

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.641e+11, tolerance: 7.723e+07
  model = cd_fast.enet_coordinate_descent(


Lasso (alpha=0.01): R² = 0.2681


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.936e+11, tolerance: 7.723e+07
  model = cd_fast.enet_coordinate_descent(


Lasso (alpha=0.05): R² = 0.2681
Lasso (alpha=0.1): R² = 0.2681

Melhor modelo Lasso: alpha = 0, R² = 0.2681

Ridge obteve melhor resultado.


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.314e+11, tolerance: 7.723e+07
  model = cd_fast.enet_coordinate_descent(


In [10]:
# 4. Modelo Stepwise
lr = LinearRegression()
sfs = SequentialFeatureSelector(lr, n_features_to_select='auto', direction='forward', cv=5)

# Imputar valores ausentes em X_train antes de ajustar o SFS
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)

sfs.fit(X_train_imputed, y_train)

X_train_sfs = sfs.transform(X_train_imputed)

# Imputar valores ausentes em X_test antes de transformar
X_test_imputed = imputer.transform(X_test)
X_test_sfs = sfs.transform(X_test_imputed)

lr.fit(X_train_sfs, y_train)
y_pred_sfs = lr.predict(X_test_sfs)
stepwise_score = r2_score(y_test, y_pred_sfs)

print(f"\nModelo Stepwise: R² = {stepwise_score:.4f}")


Modelo Stepwise: R² = 0.2691


In [11]:
# Comparação final
best_score = max(max(ridge_scores), max(lasso_scores), stepwise_score)
if best_score == max(ridge_scores):
    print(f"\nMelhor resultado geral: Ridge (alpha={best_ridge_alpha}), R² = {best_score:.4f}")
elif best_score == max(lasso_scores):
    print(f"\nMelhor resultado geral: Lasso (alpha={best_lasso_alpha}), R² = {best_score:.4f}")
else:
    print(f"\nMelhor resultado geral: Stepwise, R² = {best_score:.4f}")


Melhor resultado geral: Stepwise, R² = 0.2691


In [12]:
# Carregar e preparar os dados
df = pd.read_csv("previsao_de_renda.csv")
X = pd.get_dummies(df.drop('renda', axis=1), drop_first=True)
y = df['renda']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [13]:
# Funções auxiliares
def print_model_results(model_name, r2, coef=None):
    print(f"\n{model_name}: R² = {r2:.4f}")
    if coef is not None:
        print("Top 5 coeficientes mais importantes:")
        coef_sorted = sorted(zip(X.columns, coef), key=lambda x: abs(x[1]), reverse=True)
        for name, value in coef_sorted[:5]:
            print(f"{name}: {value:.4f}")

In [16]:
# Ridge e Lasso
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
best_ridge, best_lasso = None, None
best_ridge_r2, best_lasso_r2 = -np.inf, -np.inf

# Imputar valores ausentes em X_train_scaled antes de ajustar os modelos
imputer = SimpleImputer(strategy='mean')
X_train_scaled = imputer.fit_transform(X_train_scaled)

for alpha in alphas:
    for model_class in [Ridge, Lasso]:
        model = model_class(alpha=alpha)

        #Ajustar o modelo aos dados imputados
        model.fit(X_train_scaled, y_train)

        # Imputar valores ausentes em X_test_scaled antes de prever
        X_test_scaled = imputer.transform(X_test_scaled)

        y_pred = model.predict(X_test_scaled)
        r2 = r2_score(y_test, y_pred)

        if model_class == Ridge and r2 > best_ridge_r2:
            best_ridge, best_ridge_r2 = model, r2
        elif model_class == Lasso and r2 > best_lasso_r2:
            best_lasso, best_lasso_r2 = model, r2

print_model_results("Best Ridge", best_ridge_r2, best_ridge.coef_)
print_model_results("Best Lasso", best_lasso_r2, best_lasso.coef_)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.855e+11, tolerance: 7.723e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent


Best Ridge: R² = 0.2681
Top 5 coeficientes mais importantes:
tempo_emprego: 3728.8335
sexo_M: 2733.2979
qt_pessoas_residencia: 1025.8152
Unnamed: 0: -895.9723
qtd_filhos: -769.0141

Best Lasso: R² = 0.2681
Top 5 coeficientes mais importantes:
tempo_emprego: 3728.8443
sexo_M: 2733.1818
qt_pessoas_residencia: 1018.1999
Unnamed: 0: -768.4793
qtd_filhos: -762.7691


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.314e+11, tolerance: 7.723e+07
  model = cd_fast.enet_coordinate_descent(


In [17]:
# Stepwise
lr = LinearRegression()
# Imputar valores ausentes em X_train antes de ajustar o SFS
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)

sfs = SequentialFeatureSelector(lr, n_features_to_select='auto', direction='forward', cv=5)
sfs.fit(X_train_imputed, y_train)

X_train_sfs = sfs.transform(X_train_imputed)
# Imputar valores ausentes em X_test antes de transformar
X_test_imputed = imputer.transform(X_test)
X_test_sfs = sfs.transform(X_test_imputed)
lr.fit(X_train_sfs, y_train)
y_pred_sfs = lr.predict(X_test_sfs)
stepwise_r2 = r2_score(y_test, y_pred_sfs)
print_model_results("Stepwise", stepwise_r2, lr.coef_)


Stepwise: R² = 0.2691
Top 5 coeficientes mais importantes:
data_ref_2015-02-01: 5839.0880
data_ref_2015-05-01: -1452.7469
data_ref_2015-03-01: -1075.1550
data_ref_2015-06-01: 999.9734
data_ref_2015-04-01: 862.6125


In [18]:
# 6. Melhorando o modelo
poly = PolynomialFeatures(degree=2, include_bias=False)
# Imputar valores ausentes em X_train antes de ajustar
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_train_poly = poly.fit_transform(X_train_imputed)

# Imputar valores ausentes em X_test antes de transformar
X_test_imputed = imputer.transform(X_test)
X_test_poly = poly.transform(X_test_imputed)

ridge_poly = Ridge(alpha=0.1)
ridge_poly.fit(X_train_poly, y_train)
y_pred_poly = ridge_poly.predict(X_test_poly)
poly_r2 = r2_score(y_test, y_pred_poly)
print_model_results("Polynomial Ridge", poly_r2)


Polynomial Ridge: R² = 0.3532


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=3.27705e-22): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


In [19]:
# 7. Árvore de Regressão
tree = DecisionTreeRegressor(random_state=42)
tree.fit(X_train, y_train)
y_pred_tree = tree.predict(X_test)
tree_r2 = r2_score(y_test, y_pred_tree)
print_model_results("Decision Tree", tree_r2)


Decision Tree: R² = 0.1438


In [20]:
# Random Forest (como bônus)
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Imputar valores ausentes em X_train antes de ajustar o modelo
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)

# Ajustar o modelo aos dados imputados
rf.fit(X_train_imputed, y_train)

# Imputar valores ausentes em X_test antes de prever
X_test_imputed = imputer.transform(X_test)

y_pred_rf = rf.predict(X_test_imputed)
rf_r2 = r2_score(y_test, y_pred_rf)
print_model_results("Random Forest", rf_r2)


Random Forest: R² = 0.4159


In [21]:
# Comparação final
models = {
    "Ridge": best_ridge_r2,
    "Lasso": best_lasso_r2,
    "Stepwise": stepwise_r2,
    "Polynomial Ridge": poly_r2,
    "Decision Tree": tree_r2,
    "Random Forest": rf_r2
}

best_model = max(models, key=models.get)
print(f"\nMelhor modelo: {best_model} com R² = {models[best_model]:.4f}")

print("\nMinha opinião sobre o melhor modelo:")
print("Considerando o equilíbrio entre desempenho (R²) e interpretabilidade, eu escolheria...")


Melhor modelo: Random Forest com R² = 0.4159

Minha opinião sobre o melhor modelo:
Considerando o equilíbrio entre desempenho (R²) e interpretabilidade, eu escolheria...
